### Simple Graph

In [24]:
import tensorflow as tf
x=tf.Variable(3.0, name="x", dtype=tf.float32)
y=tf.Variable(4.0, name="y", dtype=tf.float32)

f = x*x*y+y+2

- Above statement only defines tensorflow graph it doesn't yet initialize variables or do calculations
- x,y and f are nodes and tensorflow graph. Which are automatically added to default graph

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
sess.close()
result

42.0

### Using "with" Block

In [9]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

### Initializing Global Variables At Once

In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
result

42

### Interactive Session
- To do experiment with one session, need to call close after done

In [11]:
in_sess = tf.InteractiveSession() # Crete and set session as default
init.run()
result = f.eval()
in_sess.close()
result


42

### Gradient Descent Using Tensorflow

In [1]:
from sklearn.datasets import fetch_california_housing
from pandas import DataFrame
import numpy as np
from sklearn.preprocessing import StandardScaler


housing_data = fetch_california_housing()
m,n = housing_data.data.shape
scaled_data = StandardScaler().fit_transform(housing_data.data)
housing_data_df = DataFrame(scaled_data, columns=housing_data.feature_names)
housing_data_df["bais"] = np.ones(m)
columns = ["bais"] + list(housing_data_df.columns[:-1])
housing_data_df = housing_data_df[columns]
housing_data_df.head()

bais    MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  \
0   1.0  2.344766  0.982143  0.628559  -0.153758   -0.974429 -0.049597   
1   1.0  2.332238 -0.607019  0.327041  -0.263336    0.861439 -0.092512   
2   1.0  1.782699  1.856182  1.155620  -0.049016   -0.820777 -0.025843   
3   1.0  0.932968  1.856182  0.156966  -0.049833   -0.766028 -0.050329   
4   1.0 -0.012881  1.856182  0.344711  -0.032906   -0.759847 -0.085616   

   Latitude  Longitude  
0  1.052548  -1.327835  
1  1.043185  -1.322844  
2  1.038503  -1.332827  
3  1.038503  -1.337818  
4  1.038503  -1.337818

In [43]:
import tensorflow as tf

lr = 0.01

X = tf.constant(housing_data_df.values, dtype=tf.float32, name="X")
y = tf.constant(housing_data.target.reshape(-1,1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n+1,1], -1.0,1.0,dtype=
                                     tf.float32), dtype=tf.float32, name="theta")
preds_op = tf.matmul(X,theta,name="predictions")
error_op = preds_op - y
grad_op = (2/m) * tf.matmul(tf.transpose(X), error_op, name="gradients")
training_op = tf.assign(theta, theta-lr*grad_op)

mse_op = tf.reduce_mean(tf.square(error_op), name="mse")
init_op = tf.global_variables_initializer()

In [39]:
def perform_gradient_descent(training_op, n_epoch=1000):
    with tf.Session() as sess:
        ## The line below is added to avoid error 
        ## "FailedPreconditionError: Attempting to use uninitialized value theta_23"
        ## when using MomentumOptimizer
#         tf.initialize_all_variables().run()
        sess.run(init_op)
        theta.eval()
        with MSE_Graph_Saver(mse_op) as graph_saver:
            for epoch in range(n_epoch):
                sess.run(training_op)
                if epoch%100==0:
                    print(mse_op.eval())
                graph_saver.log_summary(epoch, None)
        best_theta = theta.eval()
        print(best_theta)

In [44]:
perform_gradient_descent(training_op)

'__enter__'

7.6266003
0.7294788
0.578965
0.5592433
0.54923034
0.54227024
0.53726745
0.53366005
0.53105795
0.5291809


'__exit__'

[[ 2.0685523 ]
 [ 0.7962932 ]
 [ 0.13590078]
 [-0.15879823]
 [ 0.19819374]
 [ 0.00216718]
 [-0.04006612]
 [-0.80615646]
 [-0.77081573]]


### Using Auto Diff
- Auto diff calculates the gradient numerically with respect to given variable

In [108]:
auto_diff_grad = tf.gradients(mse_op,[theta])[0]
auto_diff_train_op = tf.assign(theta, theta-lr*auto_diff_grad)

perform_gradient_descent(auto_diff_train_op)

6.668069
0.7467046
0.63447845
0.6107754
0.5934411
0.5798341
0.56907713
0.560532
0.55371314
0.548249
[[ 2.0685525 ]
 [ 0.9469755 ]
 [ 0.16062963]
 [-0.45007995]
 [ 0.44195756]
 [ 0.00920807]
 [-0.04537104]
 [-0.484403  ]
 [-0.46691895]]


### Using Optimizer
- Optimizer automatically performs gradient descent on cost function operator
- It automatically detects the variables in the cost function and change it to reduce the cost function

#### Gradient Descent Optimiser

In [107]:
tf.train.GradientDescentOptimizer(learning_rate=lr)
gd_optimiser_train_op = optimizer.minimize(mse_op)
perform_gradient_descent(gd_optimiser_train_op)

7.648159
0.7675881
0.6371839
0.60500616
0.5829837
0.5670564
0.55549943
0.5471045
0.5409999
0.5365559
[[ 2.0685525 ]
 [ 0.8178526 ]
 [ 0.15121251]
 [-0.17845586]
 [ 0.2051815 ]
 [ 0.00745886]
 [-0.04182141]
 [-0.67430985]
 [-0.64045894]]


#### Momentum Optimizer

In [124]:
m_optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.75)
momentum_optimizer_train_op = m_optimizer.minimize(mse_op)
perform_gradient_descent(momentum_optimizer_train_op)

3.8320448
0.5571656
0.5362444
0.5290273
0.5263092
0.52520233
0.5247241
0.5245088
0.5244095
0.5243629
[[ 2.0685587 ]
 [ 0.83449966]
 [ 0.11978285]
 [-0.27453005]
 [ 0.3130455 ]
 [-0.00419122]
 [-0.03951756]
 [-0.8877857 ]
 [-0.8589985 ]]


### Minibatch Gradient Descent

In [2]:

def fetch_minibatch(batch_size):
    n_batches =  m//batch_size
    rand_index = np.random.permutation(m)
    X = housing_data_df.values
    for batch in range(n_batches):
        cur_index = rand_index[batch*batch_size:(batch+1)*batch_size]
        y = housing_data.target.reshape(-1,1)[cur_index, :]
        yield (X[cur_index,:], y)
    
    
    

In [51]:
import tensorflow as tf
def perform_minibatch_gd(n_epoch=500):
    lr=0.001
    batch_size=1000
    ## placeholder is used to send parameters when running the operations
    X_mini = tf.placeholder(dtype=tf.float32, shape=(None,  n+1), name="X")
    y_mini = tf.placeholder(dtype=tf.float32, shape=(None,  1), name="y")

    theta = tf.Variable(tf.random_uniform([n+1,1], -1.0,1.0,dtype=
                                         tf.float32), dtype=tf.float32, name="theta")
    with tf.name_scope("gradient") as scope:
        preds_op = tf.matmul(X_mini,theta,name="predict")
        error_op = preds_op - y_mini
        grad_op = (2/batch_size) * tf.matmul(tf.transpose(X_mini), 
                                             error_op, name="find_gradient")
    
    with tf.name_scope("step") as scope:
        training_op = tf.assign(theta, theta-lr*grad_op)
    
    with tf.name_scope("loss") as scope:
        mse_mini = tf.reduce_mean(tf.square(error_op))
    
    
    
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        
        with MSE_Graph_Saver(mse_mini) as graph_saver:
            for epoch in range(n_epoch):
                for x,y in fetch_minibatch(batch_size):
                    feed_dict={X_mini:x, y_mini:y}
                    sess.run(training_op, feed_dict=feed_dict)
                    
                
                mse_str = graph_saver.log_summary(epoch, feed_dict)
                    
            best_theta = theta.eval()
    
    return best_theta
                

### Log Graph Summary
- Following class logs graph summary in a directory which can be viewed using tensorboard command

```
tensorboard --logdir <dir path>
```


In [28]:
from datetime import datetime

class MSE_Graph_Saver():
    def __init__(self, mse):
        root_dir = "tf_logs"
        now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        self.log_dir = "{}/run_{}/".format(root_dir, now)
        self.mse_summary = tf.summary.scalar("MSE", mse)
        
        
    def __enter__(self):
        self.file_writer = tf.summary.FileWriter(self.log_dir, tf.get_default_graph())
        display("__enter__")
        return self
    
    def __exit__(self, type_, value, traceback):
        self.file_writer.close()
        display ("__exit__")
        
    def log_summary(self,step, feed_dict):
        mse_str = self.mse_summary.eval(feed_dict=feed_dict)
        self.file_writer.add_summary(mse_str, step)
        return mse_str
    

In [53]:
perform_minibatch_gd()

'__enter__'

'__exit__'

array([[ 2.0686114 ],
       [ 0.7755447 ],
       [ 0.15780748],
       [-0.07150789],
       [ 0.10519094],
       [ 0.01044764],
       [-0.04112587],
       [-0.66566163],
       [-0.62580633]], dtype=float32)

In [52]:
tf.reset_default_graph()